In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Desktop\\endtoend-mlops\\projects\\covid_chest_classifier\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\Desktop\\endtoend-mlops\\projects\\covid_chest_classifier'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Yusuf-Abol/endtoend-mlops.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Yusuf-Abol"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1ed48983c97e28f11ce699668993977aeca9bdb3"

In [6]:
import torch
import torch.nn as nn

In [7]:
model = torch.load("artifacts/training/model.pth", weights_only=False)

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from ccclassifier.constants import *
from ccclassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.pth",
            training_data="artifacts/data_ingestion/Covid19-dataset",
            mlflow_uri="https://dagshub.com/Yusuf-Abol/endtoend-mlops.mlflow",
            all_params=self.params,
            params_image_size=self.params.base_model.image_size + [3],  # Add channel dimension
            params_batch_size=self.params.training.batch_size
        )
        return eval_config

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import mlflow
import mlflow.pytorch
from pathlib import Path
from urllib.parse import urlparse
import json

[2025-09-20 11:51:16,463: INFO: utils: NumExpr defaulting to 8 threads.]


In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    def _valid_generator(self):
        transform = transforms.Compose([
            transforms.Resize(self.config.params_image_size[:-1]),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        full_dataset = datasets.ImageFolder(
            root=self.config.training_data,
            transform=transform
        )
        
        train_size = int(0.7 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        _, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])
        
        self.valid_generator = DataLoader(
            val_dataset,
            batch_size=self.config.params_batch_size,
            shuffle=False
        )

    @staticmethod
    def load_model(path: Path) -> nn.Module:
        # Load state dict and create model architecture
        state_dict = torch.load(path, map_location='cpu', weights_only=False)
        
        # You need to recreate your model architecture here
        # Example for ResNet18:
        from torchvision.models import resnet18
        model = resnet18(num_classes=3)  # Adjust num_classes as needed
        model.load_state_dict(state_dict)
        
        return model
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.model.to(self.device)
        self._valid_generator()
        
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        criterion = nn.CrossEntropyLoss()
        
        with torch.no_grad():
            for data, target in self.valid_generator:
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                loss = criterion(output, target)
                total_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        
        avg_loss = total_loss / len(self.valid_generator)
        accuracy = correct / total
        self.score = [avg_loss, accuracy]
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.pytorch.log_model(self.model, "model", registered_model_name="ResNet18Model")
            else:
                mlflow.pytorch.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-09-20 11:51:20,062: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-20 11:51:20,067: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-20 11:51:20,070: INFO: common: created directory at: artifacts]
[2025-09-20 11:51:28,230: INFO: common: json file saved at: scores.json]
[2025-09-20 11:52:27,744: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Yusuf-Abol/endtoend-mlops.mlflow/api/2.0/mlflow/runs/create]
[2025-09-20 11:52:44,746: WARNING: connectionpool: Retrying (Retry(total=3, connect=5, read=3, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Yusuf-Abol/endtoend-mlops.mlflow/api/2.0/mlflow/runs/create]
[2025-09-20 11:53:05,578: WARNING: connectionpool: Retrying (Retry(total=2, connect=5, read=2, redirect=5, status=5)) after

2025/09/20 11:53:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ResNet18Model' already exists. Creating a new version of this model...
2025/09/20 11:57:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ResNet18Model, version 5
Created version '5' of model 'ResNet18Model'.


🏃 View run sneaky-fly-606 at: https://dagshub.com/Yusuf-Abol/endtoend-mlops.mlflow/#/experiments/0/runs/75af3eefad6746168ef30e60e20efadb
🧪 View experiment at: https://dagshub.com/Yusuf-Abol/endtoend-mlops.mlflow/#/experiments/0
